In [ ]:
import os

import numpy as np
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from PIL import Image

from unet import get_unet, jaccard_coef_int, jaccard_coef_loss

# LOAD THE MODEL

In [ ]:
# set up the gpu id# set u 
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [ ]:
model = get_unet(3, 512, 512)

In [ ]:
adam = Adam(lr=1e-3)
model.compile(adam, loss=jaccard_coef_loss, metrics=['binary_crossentropy', jaccard_coef_int])

# DEFINE GENERATOR

In [ ]:
import json
import random
from pycocotools.mask import decode
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from skimage.transform import resize

In [ ]:
labels = json.load(open('/root/data/blender_v2/training/free_low_rez/labels.json'))

In [ ]:
test = decode(labels[0]['masks'])

In [ ]:
plt.imshow(test[...,0])

In [ ]:
test[...,0].max()

In [ ]:
batch_size = 8
steps_per_epoch = len(labels) // batch_size
input_shape = (512, 512, 3)

In [ ]:
def build_truth(masks, input_shape):
    masks = decode(masks)
    mask = np.sum(masks, axis=2)
    # return mask[x:x+input_shape[0], y:y+input_shape[1]]
    mask = resize(mask, (input_shape[0], input_shape[1]))
    mask[mask>0] = 1
    return mask

In [ ]:
def generator(labels, steps_per_epoch, BATCH_SIZE, input_shape):
    i = 0
    img_size = input_shape[0]
    while True:
        x_batch = np.empty((BATCH_SIZE, input_shape[0], input_shape[1], input_shape[2]), dtype=np.uint8)
        y_batch = np.empty((BATCH_SIZE, input_shape[0], input_shape[1], 1), dtype=np.uint8)
        for (ind, j) in enumerate(range(i*BATCH_SIZE, (i+1)*BATCH_SIZE)):
            lab = np.random.choice(labels)
#             x = 100
#             y = 100
            x_batch[ind,...] = np.array(Image.open(lab['path']).resize((input_shape[0], input_shape[0])))
            y_batch[ind,...] = np.expand_dims(build_truth(lab['masks'], input_shape), axis =2)
        i += 1
        if i >= steps_per_epoch:
            i = 0
        yield x_batch, y_batch

In [ ]:
x = 100
y = 100
np.array(Image.open(labels[0]['path']))[x:x+input_shape[0], y:y+input_shape[1]]

In [ ]:
train_generator = generator(labels, steps_per_epoch, batch_size, input_shape)
validation_generator = generator(labels, steps_per_epoch, batch_size, input_shape)

In [ ]:
xb, yb = train_generator.next()

In [ ]:
xb.dtype

In [ ]:
plt.imshow(xb[0,...])

In [ ]:
xb.dtype

In [ ]:
plt.imshow(yb[0,...,0])
plt.colorbar()

# TRAIN

In [ ]:
# start training# start 
history = model.fit_generator(
        generator=train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=30,
        verbose=1,
        validation_data=validation_generator,
        validation_steps=steps_per_epoch//10)

# PREDICTION

In [ ]:
from PIL import Image
import numpy as np

In [ ]:
imp = np.array(Image.open('/root/data/blender_v2/training/occlusion_low_rez/collage_0.jpeg').resize((512, 512)))

In [ ]:
plt.imshow(imp)

In [ ]:
out = model.predict(np.expand_dims(imp, axis=0))

In [ ]:
out.shape

In [ ]:
plt.imshow(out[0,...,0])
plt.colorbar()
# plt.clim([0, 0.0001])

In [ ]:
np.count_nonzero(out==1)

In [ ]:
import copy

In [ ]:
img = Image.fromarray(imp)
mask = np.zeros((512, 512, 3), dtype=np.uint8)
tmp = copy.deepcopy(out[0,...,0])
tmp[tmp>0.1] = 255
mask[:,:,0] = tmp
mask = Image.fromarray(mask)

In [ ]:
Image.blend(img, mask, 0.5)